<a href="https://colab.research.google.com/github/Ray7788/FT5005-Group6/blob/main/Cleaned%20Data/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/FT5005/Cleaned\ Data/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/FT5005/Cleaned Data


In [2]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
processed_df = pd.read_csv("EBITDA_new.csv")
processed_df.head(3)

,ticker,fiscalQuarter,commonSharesOutstanding_,commonSharesTraded_,capitalExpenditure_,netIncome_,operatingIncome_,shareholdersEquity_,totalAssets_,EPS_,...,EBITDA,salePrice,realGDPSA,m2SA,m2m1GrowthGap,m2Velocity,primeRate,EBITDA_lag1,year,quarter
0,0,2013Q1,-0.000933,12.247579,0.0,-0.073272,-0.095742,-0.003005,0.0,0.0,...,50.369,13.564960,-0.867729,9.261987,0.172473,1.581,3.25,NaN,2013,1
1,0,2013Q2,-0.000933,12.247579,0.0,-0.073210,-0.095754,-0.003005,0.0,0.0,...,42.617,13.651304,-0.849252,9.272702,-0.640550,1.572,3.25,50.369,2013,2
2,0,2013Q3,-0.000933,12.247579,0.0,-0.073519,-0.095692,-0.003005,0.0,0.0,...,46.147,13.650265,-0.790315,9.286502,-0.328302,1.571,3.25,42.617,2013,3


In [6]:
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb

# Select features and target variable
# features exclude:
features = [col for col in processed_df.columns if col not in [ 'EBITDA', 'fiscalQuarter']]
target = 'EBITDA'

# Remove missing values
processed_df = processed_df.dropna(subset=features + [target])

# Split the dataset based on year

train_df = processed_df[processed_df['year'] <= 2020]
test_df = processed_df[processed_df['year'] > 2020]

# Update train and test sets
X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]
# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'verbose': -1
}

# Train the model
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, test_data], num_boost_round=100)

# Make predictions
y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

RMSE: 208.7180942776796
R2 Score: 0.9676908431145301


In [7]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 创建XGBoost模型
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.1,
    max_depth=6,
    n_estimators=1000,
    early_stopping_rounds=50,
    verbosity=1
)

# 训练模型
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    # eval_metric='rmse',
    # verbose=100
)

# 预测
y_pred_xgb = xgb_model.predict(X_test)

# 评估模型
rmse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost RMSE: {rmse_xgb}")
print(f"XGBoost R2 Score: {r2_xgb}")

[0]	validation_0-rmse:68.17862	validation_1-rmse:72.97890
[1]	validation_0-rmse:61.60848	validation_1-rmse:66.16905
[2]	validation_0-rmse:55.69505	validation_1-rmse:60.28874
[3]	validation_0-rmse:50.34266	validation_1-rmse:54.88415
[4]	validation_0-rmse:45.51241	validation_1-rmse:50.02833
[5]	validation_0-rmse:41.15149	validation_1-rmse:45.67956
[6]	validation_0-rmse:37.21607	validation_1-rmse:41.90996
[7]	validation_0-rmse:33.66935	validation_1-rmse:38.46768
[8]	validation_0-rmse:30.47401	validation_1-rmse:35.41859
[9]	validation_0-rmse:27.59320	validation_1-rmse:32.67406
[10]	validation_0-rmse:24.98570	validation_1-rmse:30.33502
[11]	validation_0-rmse:22.62754	validation_1-rmse:28.11714
[12]	validation_0-rmse:20.51025	validation_1-rmse:26.17450
[13]	validation_0-rmse:18.59227	validation_1-rmse:24.52615
[14]	validation_0-rmse:16.85720	validation_1-rmse:23.13984
[15]	validation_0-rmse:15.29023	validation_1-rmse:21.86740
[16]	validation_0-rmse:13.87717	validation_1-rmse:20.78890
[17]	va